In [1]:
import random
class Sentiment:
    NEGATIVE = 'NEGATIVE'
    NEUTRAL = 'NEUTRAL'
    POSITIVE = 'POSITIVE'
    
class Review:
    def __init__(self, text, score):
        self.text= text
        self.score= score
        self.sentiment= self.get_sentiment()
    
    def get_sentiment(self):
        if(self.score<=2):
            return Sentiment.NEGATIVE
        elif(self.score ==3):
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE

class ReviewContainer:
    def __init__(self, reviews):
        self.reviews=reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews= negative + positive_shrunk
        random.shuffle(self.reviews)

In [2]:
import json

file_name= './books_small_10000.json'
    
reviews= []
with open(file_name) as f:
    for line in f:
        review= json.loads(line)
        reviews.append(Review(review['reviewText'],review['overall']))

reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

In [3]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

train_container.evenly_distribute()
test_container.evenly_distribute()

In [25]:
train_x= train_container.get_text()
train_y= train_container.get_sentiment()

test_x = test_container.get_text()
test_y= test_container.get_sentiment()

train_y.count(Sentiment.POSITIVE)
train_y.count(Sentiment.NEGATIVE)

['A lot of words but little content. Warmed-over platitudes abound.Her other books disappoint in a similar manner.Save your money for Syd.',
 "The Double is definitely not one of Pelecanos's best efforts.  The story is threadbare and makes little sense as it goes from minor burglary to intense shoot outs and killings for no real reason.  The sub plots are meaningless and the dialogue is mostly laughable.  Pelecanos really struggles with writing in the present day.. he is much better with the 70's and 80's where he can show his knowledge of music and cars from those eras.  As a DC native I do enjoy the DC setting, but even that gets old and seems mostly to be filler in this book.  Look for very early Pelecanos books to read something really exciting.",
 'I love Scott Turow, especially Presumed Innocent and Innocent, as well as The Burden of Proof, but this one was really awful. Not only does it have fathers committing incest and then killing daughters, it has wives killing husbands. Bas

In [35]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import pickle
#tfidfvectorizer: words that show up many times is less cared
vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)
with open('./tfidf.pkl','wb') as f:
    pickle.dump(vectorizer, f)
#vectorizer.fit(train_x)
#train_x_vectors = vectorizer.transform(train_x)

In [6]:
#Linear SVM
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors, train_y)

test_x[0]

clf_svm.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

In [7]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

In [8]:
#Naive Bayes

from sklearn.naive_bayes import GaussianNB

clf_gnb= GaussianNB()
clf_gnb.fit(train_x_vectors.todense(), train_y)

clf_gnb.predict(test_x_vectors[0].todense())

array(['POSITIVE'], dtype='<U8')

In [9]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)
clf_log.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

In [10]:
#Evaluation

#Mean Accuracy
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors.todense(), test_y))
print(clf_log.score(test_x_vectors, test_y))

0.8076923076923077
0.6466346153846154
0.6610576923076923
0.8052884615384616


In [11]:
#F1 Scores
from sklearn.metrics import f1_score

f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])
#very good at positive, but very bad at neutral and negative

array([0.80582524, 0.80952381])

In [31]:
test_set=['cannot stop reading']
new_test= vectorizer.transform(test_set)

x=clf_svm.predict(new_test)
x

array(['POSITIVE'], dtype='<U8')

In [13]:
#Grid Search Method

from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('linear','rbf'), 'C' : (1,3,8,16,32)}

svc= svm.SVC()
clf= GridSearchCV(svc, parameters, cv=5)
clf.fit(train_x_vectors, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 3, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [14]:
clf.score(test_x_vectors, test_y)

0.8197115384615384

In [27]:
import pickle

with open('./models/sentiment_classifier.pkl','wb') as f:
    pickle.dump(clf_svm, f)

In [17]:
import pickle

In [32]:
with open('./models/sentiment_classifier.pkl','rb') as f:
    loaded_clf = pickle.load(f)

In [33]:
test_set=['cannot stop reading']
new_test= vectorizer.transform(test_set)
x=clf_svm.predict(new_test)
x

array(['POSITIVE'], dtype='<U8')

In [24]:
loaded_clf.predict_proba(test_x_vectors[1],probability = True)

AttributeError: predict_proba is not available when  probability=False